# HDF5 files containing Gaia halo data 

## Imports

In [2]:
import numpy as np

In [3]:
import tables

In [10]:
import csv

## Halo sources from the GAVO mock catalog

The following TAP query was executed using TOPCAT, accessing the Gaia GDR2 mock catalog at GAVO:

In [1]:
query = \
"""
select
    ra, dec, l, b, parallax, parallax_error, phot_g_mean_mag, teff_val, logg, mass, age, a_g_val, e_bp_min_rp_val
from
    gdr2mock.main
where
    parallax > 0
    and
    parallax_error / parallax <= 0.20
    and
    abs(sin(b))/parallax > 3
"""

and the result was stored in a CSV file (TAP_14_gdr2mock_main.csv).

### Copy the CSV data into an HDF file

An HDF5 file is not only much smaller in size, but using PyTables it will also allow for faster queries.

First the definition of a single row in our HDF5 table. Each row corresponds to one source.

In [4]:
class Source(tables.IsDescription):
    ra              = tables.Float32Col()        # Right Ascencion [deg]
    dec             = tables.Float32Col()        # Declination     [deg]
    l               = tables.Float32Col()        # Heliocentric galactic longitude [deg]
    b               = tables.Float32Col()        # Heliocentric galactic latitude [deg]
    parallax        = tables.Float32Col()        # Noiseless parallax [mas]
    parallax_error  = tables.Float32Col()        # Parallax uncertainty [mas]
    Gmag            = tables.Float32Col()        # Mean G magnitude
    Teff            = tables.Float32Col()        # Effective temperature [K]
    logg            = tables.Float32Col()        # log(g)  [cgs]
    mass            = tables.Float32Col()        # Mass [Msun]
    age             = tables.Float32Col()        # Age [Gyr]
    AG              = tables.Float32Col()        # Extinction in the G-band
    EBPminRP        = tables.Float32Col()        # Reddening of BP-RP color
    distance        = tables.Float32Col()        # Noiseless heliocentric distance [kpc]
    distance_error  = tables.Float32Col()        # Heliocentric distance uncertainty [kpc]
    GCdist          = tables.Float32Col()        # Noiseless galactocentric distance [kpc]
    GClat           = tables.Float32Col()        # Noiseless galactocentric latitude [deg]
    GClon           = tables.Float32Col()        # Noiseless galactocentric longitude [deg]

Open the HDF5 file to write, and create an (empty) table:

In [6]:
h5file = tables.open_file("halo_gdr2mock.h5", mode="w", title="Selection of Halo sources in gdr2mock")

In [7]:
table = h5file.create_table("/", "HaloSources", Source, "Source data from gdr2mock at GAVO")

Get a pointer to the Row instance of our table. This will be used to fill the table.

In [11]:
source = table.row

Open the CSV file downloaded with TOPCAT, parse all lines, and fill the HDF5 table.

In [15]:
with open("/Users/joris/Downloads/TAP_14_gdr2mock_main.csv", "rt") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for n, line in enumerate(reader):
        if n == 0: continue     # skip header line    
        source['ra']              = float(line[0])
        source['dec']             = float(line[1])
        source['l']               = float(line[2])
        source['b']               = float(line[3])
        source['parallax']        = float(line[4])
        source['parallax_error']  = float(line[5])
        source['Gmag']            = float(line[6])
        source['Teff']            = float(line[7])
        source['logg']            = float(line[8])
        source['mass']            = float(line[9])
        source['age']             = float(line[10])
        source['AG']              = float(line[11])
        source['EBPminRP']        = float(line[12])
        source['distance']        = 1/float(line[4])
        source['distance_error']  = float(line[5])/float(line[4])**2        
        
        dist = 1/float(line[4])
        l = np.deg2rad(float(line[2]))
        b = np.deg2rad(float(line[3]))
        x = 8.2 - dist * np.cos(b) * np.cos(l)
        y = dist * np.cos(b) * np.sin(l)
        z = dist * np.sin(b)
        gcdist = np.sqrt(x*x + y*y + z*z)

        source['GCdist']          = gcdist
        source['GClat']           = np.rad2deg(np.arcsin(z/gcdist))          # degrees in [-90, +90]
        source['GClon']           = 180.0 + np.rad2deg(np.arctan2(y, x))     # degrees in [0, 360]
        source.append()

It's important to flush the table!  

In [16]:
table.flush()

In [18]:
print(h5file)

halo_gdr2mock.h5 (File) 'Selection of Halo sources in gdr2mock'
Last modif.: 'Wed May 23 16:03:16 2018'
Object Tree: 
/ (RootGroup) 'Selection of Halo sources in gdr2mock'
/HaloSources (Table(4551861,)) 'Source data from gdr2mock at GAVO'



So we have about 4.5 million sources in our table.

In [19]:
h5file.close()

### Example on how to read and query the HDF5 file

In [43]:
h5file = tables.open_file('halo_gdr2mock.h5', 'r')

In [44]:
print(h5file)

halo_gdr2mock.h5 (File) 'Selection of Halo sources in gdr2mock'
Last modif.: 'Wed May 23 16:05:31 2018'
Object Tree: 
/ (RootGroup) 'Selection of Halo sources in gdr2mock'
/HaloSources (Table(4551861,)) 'Source data from gdr2mock at GAVO'



In [45]:
table = h5file.root.HaloSources

To find out the column names:

In [50]:
table.coldescrs

{'AG': Float32Col(shape=(), dflt=0.0, pos=0),
 'EBPminRP': Float32Col(shape=(), dflt=0.0, pos=1),
 'GCdist': Float32Col(shape=(), dflt=0.0, pos=2),
 'GClat': Float32Col(shape=(), dflt=0.0, pos=3),
 'GClon': Float32Col(shape=(), dflt=0.0, pos=4),
 'Gmag': Float32Col(shape=(), dflt=0.0, pos=5),
 'Teff': Float32Col(shape=(), dflt=0.0, pos=6),
 'age': Float32Col(shape=(), dflt=0.0, pos=7),
 'b': Float32Col(shape=(), dflt=0.0, pos=8),
 'dec': Float32Col(shape=(), dflt=0.0, pos=9),
 'distance': Float32Col(shape=(), dflt=0.0, pos=10),
 'distance_error': Float32Col(shape=(), dflt=0.0, pos=11),
 'l': Float32Col(shape=(), dflt=0.0, pos=12),
 'logg': Float32Col(shape=(), dflt=0.0, pos=13),
 'mass': Float32Col(shape=(), dflt=0.0, pos=14),
 'parallax': Float32Col(shape=(), dflt=0.0, pos=15),
 'parallax_error': Float32Col(shape=(), dflt=0.0, pos=16),
 'ra': Float32Col(shape=(), dflt=0.0, pos=17)}

To copy a column into a numpy array. Only do this if the data all fits into memory.

In [51]:
GCdist = table.col("GCdist")

In [34]:
print(len(GCdist))
print(GCdist)

4551861
[ 9.190434  8.852382  8.699171 ...  9.547629 11.259307 11.824063]


To query, we can also use **in-kernel searches**. Such queries are interpreted using the Numexpr package for achieving C-speed computation of array operations.

In [52]:
selection = \
"""
(Gmag > 14) & (distance * sin(b*3.14159/180) > 3)
"""

In [54]:
result = table.read_where(selection)

In [55]:
print(len(result))

21890


In [57]:
print(result[:5])

[(0.079, 0.037 , 10.559762 , 16.746853, 234.10042, 14.1855, 4408.53, 11., 19.695711, 45.211105, 9.028204, 1.7346759, 74.50563 , 1.65902, 0.88942 , 0.110764, 0.02128216, 281.63397)
 (0.079, 0.038 , 10.038416 , 17.78269 , 231.09494, 14.3129, 4486.69, 11., 21.567657, 44.943832, 8.340075, 1.6114217, 73.54691 , 1.81243, 0.942609, 0.119903, 0.02316697, 278.71863)
 (0.163, 0.0785, 10.709331 , 17.467978, 231.25261, 14.0039, 4831.21, 13., 21.479668, 48.18661 , 8.779092, 1.5072381, 77.22624 , 1.73951, 0.795574, 0.113907, 0.01955612, 280.31744)
 (0.061, 0.03  , 10.6213455, 18.584202, 231.83923, 14.0999, 4954.98, 13., 22.530931, 47.36275 , 8.833922, 1.6342727, 75.959564, 1.8627 , 0.789456, 0.1132  , 0.02094196, 278.31787)
 (0.078, 0.038 , 11.256164 , 16.04918 , 230.13618, 14.2321, 4321.27, 11., 20.329742, 51.48607 , 8.957122, 1.7912903, 81.32812 , 1.64713, 0.913898, 0.111643, 0.02232693, 283.82773)]


In [60]:
print(result["GCdist"][5])

11.039515


If you create indexes for one or more columns, you can speed up the searches even more: see http://www.pytables.org/usersguide/optimization.html

In [42]:
h5file.close()

## Halo sources from the Gaia GDR2 archive

We used again TOPCAT, this time to query Gaia archive at ESA. The number of the rows was initially larger than the maximum limit (3M), so we split up the query in two parts:

In [ ]:
query1 = \
"""
select
    ra,dec,l,b,parallax,parallax_error,phot_g_mean_mag, a_g_val
from
    gaiadr2.gaia_source
where
    b >= 0
    and
    parallax > 0
    and
    parallax_error / parallax <= 0.20
    and
    abs(sin(b))/parallax > 3
"""

which was saved in 'TAP_15_gaiadr2_source.csv' and:

In [ ]:
query2 = \
"""
select
    ra,dec,l,b,parallax,parallax_error,phot_g_mean_mag, a_g_val
from
    gaiadr2.gaia_source
where
    b < 0
    and
    parallax > 0
    and
    parallax_error / parallax <= 0.20
    and
    abs(sin(b))/parallax > 3
"""

which was saved in 'TAP_16_gaiadr2_source.csv'

### Copy the CSV data into an HDF file

Again we first define a single row corresponding to one source:

In [72]:
class Source(tables.IsDescription):
    ra              = tables.Float32Col()        # Right Ascencion [deg]
    dec             = tables.Float32Col()        # Declination     [deg]
    l               = tables.Float32Col()        # Heliocentric galactic longitude [deg]
    b               = tables.Float32Col()        # Heliocentric galactic latitude [deg]
    parallax        = tables.Float32Col()        # Noisy parallax [mas]
    parallax_error  = tables.Float32Col()        # Parallax uncertainty [mas]
    Gmag            = tables.Float32Col()        # Mean G magnitude
    distance        = tables.Float32Col()        # Noisy heliocentric distance [kpc]
    distance_error  = tables.Float32Col()        # Heliocentric distance uncertainty [kpc]
    GCdist          = tables.Float32Col()        # Noisy galactocentric distance [kpc]
    GClat           = tables.Float32Col()        # Noisy galactocentric latitude [deg]
    GClon           = tables.Float32Col()        # Noisy galactocentric longitude [deg]

Open the HDF5 file to write, and create an (empty) table:

In [73]:
h5file = tables.open_file("halo_gaiagdr2.h5", mode="w", title="Selection of Halo sources in Gaia GDR2")

In [74]:
table = h5file.create_table("/", "HaloSources", Source, "Source data from gaiadr2.gaia_source")

Get a pointer to the Row instance of our table. This will be used to fill the table.

In [75]:
source = table.row

Open the first CSV file downloaded with TOPCAT, parse all lines, and fill the HDF5 table.

In [76]:
with open("/Users/joris/Downloads/TAP_15_gaiadr2_source.csv", "rt") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for n, line in enumerate(reader):
        if n == 0: continue     # skip header line    
        source['ra']              = float(line[0])
        source['dec']             = float(line[1])
        source['l']               = float(line[2])
        source['b']               = float(line[3])
        source['parallax']        = float(line[4])
        source['parallax_error']  = float(line[5])
        source['Gmag']            = float(line[6])
        source['distance']        = 1/float(line[4])
        source['distance_error']  = float(line[5])/float(line[4])**2        
        
        dist = 1/float(line[4])
        l = np.deg2rad(float(line[2]))
        b = np.deg2rad(float(line[3]))
        x = 8.2 - dist * np.cos(b) * np.cos(l)
        y = dist * np.cos(b) * np.sin(l)
        z = dist * np.sin(b)
        gcdist = np.sqrt(x*x + y*y + z*z)

        source['GCdist']          = gcdist
        source['GClat']           = np.rad2deg(np.arcsin(z/gcdist))          # degrees in [-90, +90]
        source['GClon']           = 180.0 + np.rad2deg(np.arctan2(y, x))     # degrees in [0, 360]
        source.append()

Also add all sources from the second CSV file:

In [77]:
with open("/Users/joris/Downloads/TAP_16_gaiadr2_source.csv", "rt") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for n, line in enumerate(reader):
        if n == 0: continue     # skip header line    
        source['ra']              = float(line[0])
        source['dec']             = float(line[1])
        source['l']               = float(line[2])
        source['b']               = float(line[3])
        source['parallax']        = float(line[4])
        source['parallax_error']  = float(line[5])
        source['Gmag']            = float(line[6])
        source['distance']        = 1/float(line[4])
        source['distance_error']  = float(line[5])/float(line[4])**2        
        
        dist = 1/float(line[4])
        l = np.deg2rad(float(line[2]))
        b = np.deg2rad(float(line[3]))
        x = 8.2 - dist * np.cos(b) * np.cos(l)
        y = dist * np.cos(b) * np.sin(l)
        z = dist * np.sin(b)
        gcdist = np.sqrt(x*x + y*y + z*z)

        source['GCdist']          = gcdist
        source['GClat']           = np.rad2deg(np.arcsin(z/gcdist))          # degrees in [-90, +90]
        source['GClon']           = 180.0 + np.rad2deg(np.arctan2(y, x))     # degrees in [0, 360]
        source.append()

Don't forget to flush:

In [78]:
table.flush()

In [79]:
print(h5file)

halo_gaiagdr2.h5 (File) 'Selection of Halo sources in Gaia GDR2'
Last modif.: 'Wed May 23 17:41:54 2018'
Object Tree: 
/ (RootGroup) 'Selection of Halo sources in Gaia GDR2'
/HaloSources (Table(4230683,)) 'Source data from gaiadr2.gaia_source'



So we have 4.2 million gaia sources in our HDF5 table.

In [80]:
h5file.close()